In [1]:
!git clone -b data-augmentation --single-branch \
https://github.com/oskareriksandelin/p30gnn.git


Cloning into 'p30gnn'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 123 (delta 28), reused 114 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (123/123), 69.08 KiB | 5.31 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.8 MB/s eta 0:00:0000:01


In [3]:
!pwd
!ls


/kaggle/working
p30gnn


## LOAD THE DATA

In [4]:
!pip -q install gdown   

In [5]:
import gdown

In [6]:
folder_url = "https://drive.google.com/drive/folders/1y10KJqDBL5RhETXMChe48e93Q0oubGHn?usp=drive_link"
gdown.download_folder(folder_url, output="data", quiet=False, remaining_ok=True)

Retrieving folder contents


Processing file 19HVQoNILBqnfv7vvSdcTXirFptHNVlVX FeGd_data_POSCAR_2.zip
Processing file 15lcB99k6cYbt0HGnQj_g8QXtb3N2nCFT FeGd_data_POSCAR_3.zip
Processing file 1f__wyGmIioGC9WfimVaypb6HrmuJ06vq FeGd_data_POSCAR_4.zip
Processing file 1OmYIMRib6yYpWmxcagvhKVUVHJfoiVwh FeGd_data_POSCAR_5.zip
Processing file 1x71MDnwVG6QQ4Nm9Gan6Y2uUo3TDoIFG FeGd_data_POSCAR_6.zip
Processing file 1F4OvZB1U2P1OEOxvGdr_-6CnbpVpjrRi FeGd_data_POSCAR_7.zip
Processing file 1ybzjthY_8ZHUOzcjQ62Du9v24EW4Azct FeGd_data_POSCAR_8.zip
Processing file 1baE5wY5rFIQERD0l2tJFBK2vCtyQ2c5x FeGd_data_POSCAR_9.zip


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=19HVQoNILBqnfv7vvSdcTXirFptHNVlVX
From (redirected): https://drive.google.com/uc?id=19HVQoNILBqnfv7vvSdcTXirFptHNVlVX&confirm=t&uuid=1728aa10-a647-4c86-8f1c-62a9a9230ecf
To: /kaggle/working/data/FeGd_data_POSCAR_2.zip
100%|██████████| 278M/278M [00:04<00:00, 67.5MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=15lcB99k6cYbt0HGnQj_g8QXtb3N2nCFT
From (redirected): https://drive.google.com/uc?id=15lcB99k6cYbt0HGnQj_g8QXtb3N2nCFT&confirm=t&uuid=831ccdad-5a7b-4cdf-8365-aeb896ae8f89
To: /kaggle/working/data/FeGd_data_POSCAR_3.zip
100%|██████████| 34.7M/34.7M [00:00<00:00, 61.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1f__wyGmIioGC9WfimVaypb6HrmuJ06vq
From (redirected): https://drive.google.com/uc?id=1f__wyGmIioGC9WfimVaypb6HrmuJ06vq&confirm=t&uuid=7b549635-e1eb-4149-90ed-287b6b85

['data/FeGd_data_POSCAR_2.zip',
 'data/FeGd_data_POSCAR_3.zip',
 'data/FeGd_data_POSCAR_4.zip',
 'data/FeGd_data_POSCAR_5.zip',
 'data/FeGd_data_POSCAR_6.zip',
 'data/FeGd_data_POSCAR_7.zip',
 'data/FeGd_data_POSCAR_8.zip',
 'data/FeGd_data_POSCAR_9.zip']

In [7]:
!pwd
!ls

/kaggle/working
data  p30gnn


In [8]:
!ls data

FeGd_data_POSCAR_2.zip	FeGd_data_POSCAR_5.zip	FeGd_data_POSCAR_8.zip
FeGd_data_POSCAR_3.zip	FeGd_data_POSCAR_6.zip	FeGd_data_POSCAR_9.zip
FeGd_data_POSCAR_4.zip	FeGd_data_POSCAR_7.zip


In [9]:
import zipfile
from pathlib import Path

data_path = Path("/kaggle/working/data")

for z in data_path.glob("*.zip"):
    with zipfile.ZipFile(z) as f:
        f.extractall(data_path)
    z.unlink()  # remove the zip file

In [10]:
!ls data

FeGd_data_POSCAR_2  FeGd_data_POSCAR_4	FeGd_data_POSCAR_6  FeGd_data_POSCAR_8
FeGd_data_POSCAR_3  FeGd_data_POSCAR_5	FeGd_data_POSCAR_7  FeGd_data_POSCAR_9


## Libraries

In [ ]:
from p30gnn.FeGdSpinGNN.utils import normalize_data, compute_normalization_stats
from p30gnn.FeGdSpinGNN.FeGd_dataset import FeGdMagneticDataset


In [12]:
def normalize_dataset(dataset, stats):
    normalized_list = []
    for data in dataset:
        normalized_list.append(normalize_data(data, stats))
    return normalized_list


train_dataset = FeGdMagneticDataset(data_path, systems=[2, 3, 4, 5], cutoff_dist=0.51)
val_dataset = FeGdMagneticDataset(data_path, systems=[6, 7], cutoff_dist=0.51)
test_dataset = FeGdMagneticDataset(data_path, systems=[8, 9], cutoff_dist=0.51)

# compute normalization stats
norm_stats = compute_normalization_stats(train_dataset)

train_data_norm = normalize_dataset(train_dataset, norm_stats)
val_data_norm = normalize_dataset(val_dataset, norm_stats)
test_data_norm = normalize_dataset(test_dataset, norm_stats)

Loading systems: 100%|██████████| 2/2 [00:10<00:00,  5.00s/it]


In [ ]:
from p30gnn.FeGdSpinGNN.models import NaiveFeGdBFieldModel
from p30gnn.FeGdSpinGNN.utils import train_one_epoch, evaluate

import torch
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
from time import time

# Model - regular data

In [ ]:
# --- DATALOADERS ---
train_loader = DataLoader(train_data_norm, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_data_norm,   batch_size=16, shuffle=False)
test_loader  = DataLoader(test_data_norm,  batch_size=16, shuffle=False)

# --- MODEL ---

# Hyperparameters
hidden_dim = 64
num_layers = 2
dropout = 0.2
lr=5e-3
weight_decay=5e-5
num_epochs = 40



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = NaiveFeGdBFieldModel(
    node_in_dim=5,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout=dropout
).to(device)
#model = EquivariantGNN(node_feat_dim=5, hidden_dim=64, num_layers=3).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    patience=10,
    factor=0.5,
    min_lr=1e-6
)

# --- TRAINING ---
best_val_loss = float('inf')
train_losses = []
val_losses = []
start_time = time()
for epoch in range(1, num_epochs + 1):
    train_loss = train_one_epoch(model, train_loader, optimizer, device)
    val_loss   = evaluate(model, val_loader, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"{epoch}/{num_epochs}   Train loss: {train_loss:.6f}    Val loss: {val_loss:.6f}    LR: {optimizer.param_groups[0]['lr']:.6f}")

    scheduler.step(val_loss)

    # save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pt")

train_time = time() - start_time
print(f"Training time: {train_time/60:.2f} minutes")


In [ ]:
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# --- TEST EVALUATION ---

model.load_state_dict(torch.load("best_model.pt"))
test_loss = evaluate(model, test_loader, device)

print(f"\nFinal Test Loss: {test_loss:.6f}")

# Model - augmented data

In [ ]:
# Data augmentation: random rotations
from p30gnn.FeGdSpinGNN.data_augmentation import RandomRotationTransform
rotation_transform = RandomRotationTransform()

train_dataset = FeGdMagneticDataset(data_path, systems=[2, 3, 4, 5], cutoff_dist=0.51,
                                              transform_rotate=rotation_transform)

train_aug_data_norm = normalize_dataset(train_dataset, norm_stats)
# --- DATALOADERS ---
train_loader = DataLoader(train_aug_data_norm, batch_size=16, shuffle=True)

In [ ]:
# --- MODEL ---

# Hyperparameters
hidden_dim = 64
num_layers = 2
dropout = 0.2
lr=5e-3
weight_decay=5e-5
num_epochs = 40

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = NaiveFeGdBFieldModel(
    node_in_dim=5,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout=dropout
).to(device)
#model = EquivariantGNN(node_feat_dim=5, hidden_dim=64, num_layers=3).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    patience=10,
    factor=0.5,
    min_lr=1e-6
)

# --- TRAINING ---
best_val_loss = float('inf')
train_losses = []
val_losses = []
start_time = time()
for epoch in range(1, num_epochs + 1):
    train_loss = train_one_epoch(model, train_loader, optimizer, device)
    val_loss   = evaluate(model, val_loader, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"{epoch}/{num_epochs}   Train loss: {train_loss:.6f}    Val loss: {val_loss:.6f}    LR: {optimizer.param_groups[0]['lr']:.6f}")

    scheduler.step(val_loss)

    # save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pt")

train_time = time() - start_time
print(f"Training time: {train_time/60:.2f} minutes")


In [ ]:
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# --- TEST EVALUATION ---

model.load_state_dict(torch.load("best_model.pt"))
test_loss = evaluate(model, test_loader, device)

print(f"\nFinal Test Loss: {test_loss:.6f}")